In [1]:
from simtk.openmm import app
import simtk.openmm as mm
from simtk import unit
from sys import stdout
import time as time

# what follows is from the OpenMM script builder
# with some minor modifications...

## Simulation setup

In [2]:
# read in a starting structure for ethane and the
# corresponding force field file
pdb = app.PDBFile('butane.pdb')
forcefield = app.ForceField('butane.gaff2.xml')

# setup system by taking topology from pdb file;
# run gas phase simulation with 2 fs time step (using SHAKE)
# at 298.15 K using a Langevin thermostat (integrator) with
# coupling constant of 5.0 ps^-1
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.NoCutoff, 
                                 constraints=app.HBonds)
integrator = mm.LangevinIntegrator(298.15*unit.kelvin, 5.0/unit.picoseconds, 
                                   2.0*unit.femtoseconds)
integrator.setConstraintTolerance(1e-5)

platform = mm.Platform.getPlatformByName('Reference')
simulation = app.Simulation(pdb.topology, system, integrator, platform)
simulation.context.setPositions(pdb.positions)

## Energy minimization

This reduces the potential energy of the system before beginning dynamics. This eliminates "bad" (i.e., overly close) contacts and generally leads to more stable simulation behavior.

In [3]:
print('Minimizing...')

st = simulation.context.getState(getPositions=True,getEnergy=True)
print("Potential energy before minimization is %s" % st.getPotentialEnergy())

simulation.minimizeEnergy(maxIterations=100)

st = simulation.context.getState(getPositions=True,getEnergy=True)
print("Potential energy after minimization is %s" % st.getPotentialEnergy())

Minimizing...
Potential energy before minimization is 5.797528654036221 kJ/mol
Potential energy after minimization is 5.295941643198585 kJ/mol


## Equilibration

We will run a (very) short equilibration simulation to bring the molecule up to our desired temperature.  If this were a periodic system, we would also aim to bring the density/volume to equilibrium at the desired pressure.

In [4]:
print('Equilibrating...')

simulation.reporters.append(app.StateDataReporter(stdout, 250, step=True, 
    potentialEnergy=True, temperature=True, separator=','))
simulation.context.setVelocitiesToTemperature(150.0*unit.kelvin)
simulation.step(5000)

Equilibrating...
#"Step","Potential Energy (kJ/mole)","Temperature (K)"
250,35.062097024444945,211.58505274228852
500,35.786881351135825,200.58484988702895
750,49.057564806238034,307.81745324432507
1000,44.02890133387555,331.815706017254
1250,53.140572881853466,256.190067265853
1500,30.086844676449854,248.64607553034438
1750,44.5767483186951,252.7436171388968
2000,43.97547562839223,267.2456771730894
2250,24.63659291882071,452.42899730678755
2500,42.21678782339953,233.83832710271466
2750,39.381501643180655,275.1681270967201
3000,44.86036388002023,313.1517403867972
3250,82.03839663151916,445.9352562169752
3500,44.20467842160067,368.1460895565275
3750,48.07059865993829,288.7320418022091
4000,53.404905591024466,436.1702080039039
4250,62.64006300119624,321.21261750011
4500,44.908758398585455,317.6377886273449
4750,35.8222373104688,225.54647326791914
5000,43.824178693171355,283.9813775591581


## Production

Now we run a long MD simulation with parameters that are identical to the equilibration phase (other than simulation length, of course!).  We will also save a trajectory file (i.e., corodinats vs. time) of this simulation that we can analyze afterward using MDTraj (or other trajectory analysis tools).

In [5]:
print('Running Production...')

tinit=time.time()
simulation.reporters.clear()
# output basic simulation information below every 500000 steps/1 ns
simulation.reporters.append(app.StateDataReporter(stdout, 500000, 
    step=True, time=True, potentialEnergy=True, temperature=True, 
    speed=True, separator=','))
# write out a trajectory (i.e., coordinates vs. time) to a DCD
# file every 100 steps/0.2 ps
simulation.reporters.append(app.DCDReporter('butane_sim.dcd', 100))

# run the simulation for 2.0x10^7 steps/40 ns
simulation.step(20000000)
tfinal=time.time()
print('Done!')
print('Time required for simulation:', tfinal-tinit, 'seconds')

Running Production...
#"Step","Time (ps)","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
500000,999.9999999901769,46.97980732476568,461.66813779101926,0
1000000,1999.9999999665301,41.52183358215743,282.20631069504077,1.03e+04
1500000,2999.9999999428833,31.45197911054147,355.32239064936596,1.01e+04
2000000,3999.9999999192364,67.52978836853917,169.1960895184073,1e+04
2500000,5000.000000101135,50.977210543366965,340.3034116830316,9.9e+03
3000000,6000.000000304862,38.533308500456236,288.3907288433284,9.96e+03
3500000,7000.0000005085885,75.48487105697616,310.9162751679937,9.92e+03
4000000,8000.000000712315,55.02533295404491,272.10828087866275,9.81e+03
4500000,9000.000000916041,25.886724524864402,290.8678921279834,9.95e+03
5000000,10000.000001119768,36.73644398508894,279.4022019300137,9.97e+03
5500000,11000.000001323495,51.414780482019026,275.8323926957754,1e+04
6000000,12000.000001527222,46.22195056833969,199.273291923906,1.01e+04
6500000,13000.000001730948,49.6441740754523